# LSST Twilight Planner (Modular) — Notebook

Demonstrates the filter-aware, Moon-aware LSST twilight planner. Edit the configuration cell below to explore optional parameters.

In [4]:
# Import planner package and set up paths
import sys, os, pandas as pd
sys.path.append('./')

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths for catalog and outputs
CSV_PATH = 'data/demo_three_targets.csv'  # tiny demo catalog
OUTDIR = 'twilight_outputs'
START_DATE = '2024-01-01'  # UTC
END_DATE = '2024-01-03'    # UTC
os.makedirs(OUTDIR, exist_ok=True)
print('CSV exists:', os.path.exists(CSV_PATH))
print('Output dir:', OUTDIR)


CSV exists: True
Output dir: ../twilight_outputs


## Configure the site and planner knobs
Below we expose **all** configuration parameters for the planner.
Values shown match the package defaults and can be tweaked.


In [5]:
from twilight_planner_pkg.config import PlannerConfig

# -- Example configuration (all options shown) --
cfg = PlannerConfig(
    # Site and visibility
    lat_deg=-30.2446, lon_deg=-70.7494, height_m=2647.0,
    min_alt_deg=30.0,

    # Filters and hardware
    filters=['g','r','i','z','y'],
    carousel_capacity=5,
    start_filter=None,  # use first filter by default
    filter_change_s=120.0,
    readout_s=2.0,
    exposure_by_filter={'u':30.0,'g':15.0,'r':15.0,'i':15.0,'z':15.0,'y':15.0},
    max_filters_per_visit=1,
    sun_alt_policy=[(-18.0,-15.0,['y','z','i']), (-15.0,-12.0,['z','i','r']), (-12.0,0.0,['i','z','y'])],

    # Slew model
    slew_small_deg=3.5,
    slew_small_time_s=4.0,
    slew_rate_deg_per_s=5.25,
    slew_settle_s=1.0,

    # Moon constraints
    min_moon_sep_by_filter={'u':80.0,'g':50.0,'r':35.0,'i':30.0,'z':25.0,'y':20.0},
    require_single_time_for_all_filters=True,

    # Time caps
    per_sn_cap_s=600.0,
    morning_cap_s=1800.0,
    evening_cap_s=1800.0,
    twilight_step_min=2,
    max_sn_per_night=20,

    # Priority tracking
    priority_strategy='hybrid',
    hybrid_detections=2,
    hybrid_exposure_s=300.0,
    lc_detections=5,
    lc_exposure_s=300.0,

    # Photometry/sky (defaults)
    pixel_scale_arcsec=0.2,
    zpt1s=None,
    k_m=None,
    fwhm_eff=None,
    read_noise_e=5.0,
    gain_e_per_adu=1.0,
    zpt_err_mag=0.01,
    dark_sky_mag=None,
    twilight_delta_mag=2.5,

    # SIMLIB output (optional)
    simlib_out=None,
    simlib_survey='LSST',
    simlib_filters='grizy',
    simlib_pixsize=0.2,
    simlib_npe_pixel_saturate=1.0e6,
    simlib_photflag_saturate=4096,
    simlib_psf_unit='arcsec',
)

cfg


PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, typical_days_by_type={}, default_typical_days=30, slew_small_deg=3.0, slew_small_time_s=2.0, slew_rate_deg_per_s=3.5, slew_settle_s=2.0, readout_s=2.0, filter_change_s=120.0, filters=['g', 'r', 'i', 'z'], exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, carousel_capacity=6, twilight_step_min=2, evening_cap_s=600.0, morning_cap_s=600.0, max_sn_per_night=10, per_sn_cap_s=120.0, min_moon_sep_by_filter={'g': 30.0, 'r': 25.0, 'i': 20.0, 'z': 15.0}, require_single_time_for_all_filters=True, priority_strategy='hybrid', hybrid_detections=2, hybrid_exposure_s=300.0, lc_detections=5, lc_exposure_s=300.0, lc_phase_range=(-7.0, 20.0), ra_col=None, dec_col=None, disc_col=None, name_col=None, type_col=None)

## Run the planner
This will produce two CSVs in `OUTDIR`.
The call below passes all arguments explicitly, including the optional `verbose`.


In [6]:
"""Run the filter-aware planner.

Parameters
---------
csv_path : str
    Path to the SN catalog.
outdir : str
    Directory for outputs.
start_date, end_date : str
    Inclusive UTC date range.
cfg : PlannerConfig
    Configuration object (see previous cell).
verbose : bool
    If True, prints detailed progress.
"""
perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)

RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/3 [00:00<?, ?night/s]

2024-01-01: eligible=81 visible=62 planned_total=9
2024-01-02: eligible=85 visible=71 planned_total=7
2024-01-03: eligible=86 visible=74 planned_total=7
Wrote:
  ../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv
  ../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv
Rows: per-SN=23, nights*windows=9
Per-SN rows: 23   Nights summary rows: 9


,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,best_alt_deg,priority_score,filters,exposure_s,readout_s,filter_changes_s,slew_s,total_time_s
0,2024-01-01,morning,2023yxi,47.417777,-31.222880,2024-01-01T01:12:00+00:00,89.09,1.0,g,5.0,2.0,0.0,0.0,7.0
1,2024-01-01,morning,2023yqc,38.192499,-38.310448,2024-01-01T00:36:00+00:00,82.04,1.0,g,5.0,2.0,0.0,6.1,13.1
2,2024-01-01,morning,2023zzn,46.861701,-42.658672,2024-01-01T01:10:00+00:00,77.67,1.0,g,5.0,2.0,0.0,5.4,12.4
3,2024-01-01,morning,2023ypx,18.428125,-41.627731,2023-12-31T23:44:00+00:00,77.51,1.0,g,5.0,2.0,0.0,9.1,16.1
4,2024-01-01,morning,2023ywq,15.922583,-39.874981,2023-12-31T23:44:00+00:00,77.89,1.0,g,5.0,2.0,0.0,4.0,11.0
5,2024-01-01,morning,2023zrc,17.044547,-36.059907,2023-12-31T23:44:00+00:00,81.17,1.0,g,5.0,2.0,0.0,4.3,11.3
6,2024-01-01,evening,2024C,145.975500,-29.727481,2024-01-01T08:09:00+00:00,85.01,1.0,g,5.0,2.0,0.0,0.0,7.0
7,2024-01-01,evening,2023zce,141.587417,-31.226239,2024-01-01T08:09:00+00:00,81.23,1.0,g,5.0,2.0,0.0,4.3,11.3
8,2024-01-01,evening,2023zea,148.281780,-37.998441,2024-01-01T08:09:00+00:00,81.64,1.0,g,5.0,2.0,0.0,5.6,12.6
9,2024-01-02,morning,2023yqc,38.192499,-38.310448,2024-01-02T00:32:00+00:00,82.04,1.0,g,5.0,2.0,0.0,0.0,7.0


In [7]:
nights_df

,date,twilight_window,n_candidates,n_planned,sum_time_s,window_cap_s
0,2024-01-01,morning,6,6,70.9,600
1,2024-01-01,evening,3,3,30.9,600
2,2024-01-01,W2,1,0,0.0,0
3,2024-01-02,morning,3,3,33.3,600
4,2024-01-02,evening,4,4,42.4,600
5,2024-01-02,W2,3,0,0.0,0
6,2024-01-03,morning,3,3,37.3,600
7,2024-01-03,evening,4,4,42.4,600
8,2024-01-03,W2,3,0,0.0,0



## Inspect outputs on disk


In [8]:
import glob, os, pandas as pd

files = sorted(glob.glob(os.path.join(OUTDIR, 'lsst_twilight_*.csv')))
files


['../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv']